In [1]:
# debug.ipynb
import sys
sys.path.append('.')
from Agent.graph import load_analysis,task_definition_analysis,eda_analysis
from Agent.state import SKWorkflowState
from Agent.configuration import Configuration
from Agent.data_pre_fun import feature_importance_mutual_info
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage



In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition
tools_FI = [feature_importance_mutual_info]
# 创建简单的图，只包含第一个节点
builder = StateGraph(SKWorkflowState, config_schema=Configuration)

# 添加第一个节点
builder.add_node("load_analysis", load_analysis)
builder.add_node("task_definition_analysis", task_definition_analysis)
builder.add_node("eda_analysis", eda_analysis)
builder.add_node("tools", ToolNode(tools_FI))
# 设置图的流程：开始 -> load_analysis -> 结束
builder.add_edge(START, "load_analysis")
builder.add_edge("load_analysis", "task_definition_analysis")
builder.add_edge("task_definition_analysis", "eda_analysis")

# 正确设置条件路由
# tools_condition会返回"__tools__"或"__end__"
builder.add_conditional_edges(
    "eda_analysis",
    tools_condition,
    {
        "tools": "tools", # 如果需要调用工具，则转到tools节点
        "__end__": END        # 如果不需要调用工具，则结束
    }
)
builder.add_edge("tools", "eda_analysis")



# 编译图
graph = builder.compile(name="simple-test-graph")

# 测试数据
test_inputs = {
    "messages": [HumanMessage(content="开始测试")],
    "max_research_iterations": 3,
    "current_iteration": 0,
    "data_path": "F:\\python\\机器学习工作流\\task\\train.csv",
    "task_description": "F:\\python\\机器学习工作流\\task\\data_description.txt",
}

# 配置
config = {}

# 运行图并查看结果
result = graph.invoke(test_inputs, config=config)
print(result)


RateLimitError: Error code: 429 - {'message': 'Request was rejected due to rate limiting. Details: TPM limit reached.', 'data': None}